In [1]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models.models import all_models
from synflow import SynFlow
import wandb
from utils import apply_global_mask, apply_prune_mask, compare_model, count_model_zero_params, apply_grad_mask

/home/fw/anaconda3/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/fw/anaconda3/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/fw/anaconda3/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/fw/anaconda3/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


In [2]:
device = 'cuda:1'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [3]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='../data/cifar10', train=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='../data/cifar10', train=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..


In [4]:
net = torch.load('server_model_reorder.pt')
masks = torch.load('server_masks_reorder.pt')
# masks = torch.load('masks.pt')

In [5]:
wandb.init(
            project="synflow",
            name="FedSynflow(d)_wBN",
            )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: slimfun (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [6]:


net = net.to(device)
# if device == 'cuda':
#     net = torch.nn.DataParallel(net)
#     cudnn.benchmark = True

# if args.resume:
#     # Load checkpoint.
#     print('==> Resuming from checkpoint..')
#     assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
#     checkpoint = torch.load('./checkpoint/ckpt.pth')
#     net.load_state_dict(checkpoint['net'])
#     best_acc = checkpoint['acc']
#     start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, nesterov=True,
                      momentum=0.9, weight_decay=0.00001)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
#         apply_global_mask(net, masks)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        
        print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
#         progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                      % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
#     apply_global_mask(net, masks)
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
#             progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                          % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    
    wandb.log({"Test/Acc": acc, 'epoch': epoch})
    wandb.log({"Test/Loss": (test_loss/(batch_idx+1)), 'epoch': epoch})
#     if acc > best_acc:
#         print('Saving..')
#         state = {
#             'net': net.state_dict(),
#             'acc': acc,
#             'epoch': epoch,
#         }
# #         if not os.path.isdir('checkpoint'):
# #             os.mkdir('checkpoint')
# #         torch.save(state, './checkpoint/ckpt.pth')
#         best_acc = acc


In [7]:
def check_masks(masks):
    ret = []
    for m in masks:
        ret.append(m.to('cuda:1'))
        print(float(m.mean().to('cpu')))
    return ret
masks = check_masks(masks)

0.859375
0.08056640625
0.01904296875
0.009521484375
0.004486083984375
0.00261688232421875
0.0018157958984375
0.0028533935546875
1.0


In [8]:
def check_model(net):
    total = 0.
    pruned_c = 0.
    for layer in net.modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            masked_weights = torch.where(layer.weight.data == 0, 1, 0).sum()
            print(1 - float(masked_weights) / layer.weight.data.numel())
            print(layer.weight.data.numel() - float(masked_weights))
            pruned_c += masked_weights
            total += layer.weight.numel()
    print('pruned: {}'.format(pruned_c / total))
check_model(net)

0.859375
1485.0
0.08056640625
5940.0
0.01904296875
5616.0
0.009521484375
5616.0
0.004486083984375
5292.0
0.00261688232421875
6174.0
0.0018157958984375
4284.0
0.0028533935546875
6732.0
1.0
5120.0
pruned: 0.9949843287467957


In [9]:
from utils import *

# apply_prune_mask(net, masks)
apply_grad_mask(net, masks)

for epoch in range(start_epoch, start_epoch+100):
    train(epoch)
    test(epoch)
#     scheduler.step()


Epoch: 0
Loss: 2.299 | Acc: 12.500% (8/64)
Loss: 2.301 | Acc: 12.500% (16/128)
Loss: 2.302 | Acc: 11.979% (23/192)
Loss: 2.302 | Acc: 11.328% (29/256)
Loss: 2.302 | Acc: 11.250% (36/320)
Loss: 2.302 | Acc: 10.677% (41/384)
Loss: 2.300 | Acc: 10.268% (46/448)
Loss: 2.299 | Acc: 10.156% (52/512)
Loss: 2.296 | Acc: 10.243% (59/576)
Loss: 2.295 | Acc: 10.625% (68/640)
Loss: 2.290 | Acc: 11.506% (81/704)
Loss: 2.286 | Acc: 12.370% (95/768)
Loss: 2.282 | Acc: 12.861% (107/832)
Loss: 2.276 | Acc: 13.728% (123/896)
Loss: 2.274 | Acc: 13.854% (133/960)
Loss: 2.268 | Acc: 14.062% (144/1024)
Loss: 2.264 | Acc: 14.062% (153/1088)
Loss: 2.258 | Acc: 13.802% (159/1152)
Loss: 2.249 | Acc: 14.145% (172/1216)
Loss: 2.242 | Acc: 14.453% (185/1280)
Loss: 2.231 | Acc: 15.104% (203/1344)
Loss: 2.224 | Acc: 14.915% (210/1408)
Loss: 2.225 | Acc: 14.674% (216/1472)
Loss: 2.219 | Acc: 14.714% (226/1536)
Loss: 2.210 | Acc: 15.250% (244/1600)
Loss: 2.206 | Acc: 15.325% (255/1664)
Loss: 2.200 | Acc: 15.451% (267

Loss: 1.830 | Acc: 28.996% (4027/13888)
Loss: 1.830 | Acc: 29.021% (4049/13952)
Loss: 1.828 | Acc: 29.088% (4077/14016)
Loss: 1.828 | Acc: 29.091% (4096/14080)
Loss: 1.827 | Acc: 29.101% (4116/14144)
Loss: 1.825 | Acc: 29.209% (4150/14208)
Loss: 1.825 | Acc: 29.225% (4171/14272)
Loss: 1.823 | Acc: 29.297% (4200/14336)
Loss: 1.821 | Acc: 29.361% (4228/14400)
Loss: 1.820 | Acc: 29.363% (4247/14464)
Loss: 1.819 | Acc: 29.405% (4272/14528)
Loss: 1.817 | Acc: 29.496% (4304/14592)
Loss: 1.817 | Acc: 29.490% (4322/14656)
Loss: 1.816 | Acc: 29.538% (4348/14720)
Loss: 1.816 | Acc: 29.573% (4372/14784)
Loss: 1.815 | Acc: 29.627% (4399/14848)
Loss: 1.813 | Acc: 29.681% (4426/14912)
Loss: 1.813 | Acc: 29.728% (4452/14976)
Loss: 1.811 | Acc: 29.807% (4483/15040)
Loss: 1.811 | Acc: 29.853% (4509/15104)
Loss: 1.811 | Acc: 29.872% (4531/15168)
Loss: 1.810 | Acc: 29.904% (4555/15232)
Loss: 1.809 | Acc: 29.910% (4575/15296)
Loss: 1.808 | Acc: 29.948% (4600/15360)
Loss: 1.807 | Acc: 29.953% (4620/15424)


Loss: 1.691 | Acc: 35.306% (9558/27072)
Loss: 1.690 | Acc: 35.352% (9593/27136)
Loss: 1.689 | Acc: 35.364% (9619/27200)
Loss: 1.689 | Acc: 35.387% (9648/27264)
Loss: 1.689 | Acc: 35.422% (9680/27328)
Loss: 1.688 | Acc: 35.456% (9712/27392)
Loss: 1.687 | Acc: 35.490% (9744/27456)
Loss: 1.686 | Acc: 35.512% (9773/27520)
Loss: 1.686 | Acc: 35.539% (9803/27584)
Loss: 1.686 | Acc: 35.569% (9834/27648)
Loss: 1.685 | Acc: 35.569% (9857/27712)
Loss: 1.685 | Acc: 35.606% (9890/27776)
Loss: 1.684 | Acc: 35.643% (9923/27840)
Loss: 1.684 | Acc: 35.651% (9948/27904)
Loss: 1.684 | Acc: 35.662% (9974/27968)
Loss: 1.683 | Acc: 35.695% (10006/28032)
Loss: 1.683 | Acc: 35.706% (10032/28096)
Loss: 1.682 | Acc: 35.710% (10056/28160)
Loss: 1.682 | Acc: 35.736% (10086/28224)
Loss: 1.681 | Acc: 35.771% (10119/28288)
Loss: 1.680 | Acc: 35.800% (10150/28352)
Loss: 1.680 | Acc: 35.797% (10172/28416)
Loss: 1.680 | Acc: 35.811% (10199/28480)
Loss: 1.679 | Acc: 35.825% (10226/28544)
Loss: 1.679 | Acc: 35.840% (102

Loss: 1.593 | Acc: 39.550% (15972/40384)
Loss: 1.593 | Acc: 39.540% (15993/40448)
Loss: 1.593 | Acc: 39.549% (16022/40512)
Loss: 1.592 | Acc: 39.575% (16058/40576)
Loss: 1.592 | Acc: 39.584% (16087/40640)
Loss: 1.591 | Acc: 39.618% (16126/40704)
Loss: 1.591 | Acc: 39.609% (16148/40768)
Loss: 1.591 | Acc: 39.623% (16179/40832)
Loss: 1.590 | Acc: 39.642% (16212/40896)
Loss: 1.590 | Acc: 39.653% (16242/40960)
Loss: 1.589 | Acc: 39.674% (16276/41024)
Loss: 1.589 | Acc: 39.686% (16306/41088)
Loss: 1.589 | Acc: 39.704% (16339/41152)
Loss: 1.588 | Acc: 39.713% (16368/41216)
Loss: 1.588 | Acc: 39.736% (16403/41280)
Loss: 1.588 | Acc: 39.742% (16431/41344)
Loss: 1.587 | Acc: 39.775% (16470/41408)
Loss: 1.587 | Acc: 39.781% (16498/41472)
Loss: 1.587 | Acc: 39.782% (16524/41536)
Loss: 1.586 | Acc: 39.798% (16556/41600)
Loss: 1.586 | Acc: 39.840% (16599/41664)
Loss: 1.585 | Acc: 39.884% (16643/41728)
Loss: 1.584 | Acc: 39.907% (16678/41792)
Loss: 1.584 | Acc: 39.923% (16710/41856)
Loss: 1.583 | Ac

Loss: 1.346 | Acc: 50.810% (2947/5800)
Loss: 1.348 | Acc: 50.763% (2995/5900)
Loss: 1.348 | Acc: 50.717% (3043/6000)
Loss: 1.346 | Acc: 50.770% (3097/6100)
Loss: 1.347 | Acc: 50.726% (3145/6200)
Loss: 1.346 | Acc: 50.762% (3198/6300)
Loss: 1.346 | Acc: 50.844% (3254/6400)
Loss: 1.346 | Acc: 50.815% (3303/6500)
Loss: 1.347 | Acc: 50.788% (3352/6600)
Loss: 1.348 | Acc: 50.687% (3396/6700)
Loss: 1.349 | Acc: 50.706% (3448/6800)
Loss: 1.349 | Acc: 50.710% (3499/6900)
Loss: 1.353 | Acc: 50.571% (3540/7000)
Loss: 1.353 | Acc: 50.521% (3587/7100)
Loss: 1.351 | Acc: 50.583% (3642/7200)
Loss: 1.351 | Acc: 50.589% (3693/7300)
Loss: 1.349 | Acc: 50.622% (3746/7400)
Loss: 1.351 | Acc: 50.573% (3793/7500)
Loss: 1.349 | Acc: 50.671% (3851/7600)
Loss: 1.349 | Acc: 50.649% (3900/7700)
Loss: 1.348 | Acc: 50.782% (3961/7800)
Loss: 1.347 | Acc: 50.810% (4014/7900)
Loss: 1.347 | Acc: 50.825% (4066/8000)
Loss: 1.345 | Acc: 50.827% (4117/8100)
Loss: 1.344 | Acc: 50.793% (4165/8200)
Loss: 1.345 | Acc: 50.807

...

Loss: 0.562 | Acc: 80.360% (27618/34368)
Loss: 0.562 | Acc: 80.367% (27672/34432)
Loss: 0.562 | Acc: 80.375% (27726/34496)
Loss: 0.562 | Acc: 80.379% (27779/34560)
Loss: 0.562 | Acc: 80.378% (27830/34624)
Loss: 0.562 | Acc: 80.365% (27877/34688)
Loss: 0.562 | Acc: 80.372% (27931/34752)
Loss: 0.562 | Acc: 80.362% (27979/34816)
Loss: 0.563 | Acc: 80.356% (28028/34880)
Loss: 0.563 | Acc: 80.329% (28070/34944)
Loss: 0.564 | Acc: 80.316% (28117/35008)
Loss: 0.564 | Acc: 80.312% (28167/35072)
Loss: 0.563 | Acc: 80.314% (28219/35136)
Loss: 0.563 | Acc: 80.307% (28268/35200)
Loss: 0.563 | Acc: 80.320% (28324/35264)
Loss: 0.563 | Acc: 80.322% (28376/35328)
Loss: 0.563 | Acc: 80.320% (28427/35392)
Loss: 0.563 | Acc: 80.328% (28481/35456)
Loss: 0.563 | Acc: 80.329% (28533/35520)
Loss: 0.563 | Acc: 80.328% (28584/35584)
Loss: 0.562 | Acc: 80.347% (28642/35648)
Loss: 0.563 | Acc: 80.348% (28694/35712)
Loss: 0.563 | Acc: 80.344% (28744/35776)
Loss: 0.563 | Acc: 80.335% (28792/35840)
Loss: 0.562 | Ac

Loss: 0.564 | Acc: 80.210% (38090/47488)
Loss: 0.565 | Acc: 80.203% (38138/47552)
Loss: 0.564 | Acc: 80.206% (38191/47616)
Loss: 0.564 | Acc: 80.216% (38247/47680)
Loss: 0.565 | Acc: 80.209% (38295/47744)
Loss: 0.564 | Acc: 80.213% (38348/47808)
Loss: 0.564 | Acc: 80.212% (38399/47872)
Loss: 0.565 | Acc: 80.211% (38450/47936)
Loss: 0.564 | Acc: 80.215% (38503/48000)
Loss: 0.564 | Acc: 80.218% (38556/48064)
Loss: 0.564 | Acc: 80.217% (38607/48128)
Loss: 0.564 | Acc: 80.215% (38657/48192)
Loss: 0.565 | Acc: 80.208% (38705/48256)
Loss: 0.565 | Acc: 80.203% (38754/48320)
Loss: 0.565 | Acc: 80.200% (38804/48384)
Loss: 0.565 | Acc: 80.201% (38856/48448)
Loss: 0.565 | Acc: 80.209% (38911/48512)
Loss: 0.565 | Acc: 80.196% (38956/48576)
Loss: 0.565 | Acc: 80.199% (39009/48640)
Loss: 0.565 | Acc: 80.199% (39060/48704)
Loss: 0.565 | Acc: 80.194% (39109/48768)
Loss: 0.565 | Acc: 80.202% (39164/48832)
Loss: 0.565 | Acc: 80.199% (39214/48896)
Loss: 0.565 | Acc: 80.208% (39270/48960)
Loss: 0.564 | Ac

Loss: 0.568 | Acc: 79.379% (4115/5184)
Loss: 0.568 | Acc: 79.383% (4166/5248)
Loss: 0.568 | Acc: 79.330% (4214/5312)
Loss: 0.568 | Acc: 79.334% (4265/5376)
Loss: 0.569 | Acc: 79.338% (4316/5440)
Loss: 0.569 | Acc: 79.379% (4369/5504)
Loss: 0.572 | Acc: 79.364% (4419/5568)
Loss: 0.573 | Acc: 79.332% (4468/5632)
Loss: 0.574 | Acc: 79.266% (4515/5696)
Loss: 0.572 | Acc: 79.306% (4568/5760)
Loss: 0.571 | Acc: 79.344% (4621/5824)
Loss: 0.573 | Acc: 79.314% (4670/5888)
Loss: 0.573 | Acc: 79.385% (4725/5952)
Loss: 0.572 | Acc: 79.438% (4779/6016)
Loss: 0.571 | Acc: 79.441% (4830/6080)
Loss: 0.572 | Acc: 79.460% (4882/6144)
Loss: 0.571 | Acc: 79.365% (4927/6208)
Loss: 0.571 | Acc: 79.385% (4979/6272)
Loss: 0.571 | Acc: 79.340% (5027/6336)
Loss: 0.572 | Acc: 79.266% (5073/6400)
Loss: 0.574 | Acc: 79.162% (5117/6464)
Loss: 0.576 | Acc: 79.075% (5162/6528)
Loss: 0.576 | Acc: 79.096% (5214/6592)
Loss: 0.576 | Acc: 79.102% (5265/6656)
Loss: 0.576 | Acc: 79.122% (5317/6720)
Loss: 0.576 | Acc: 79.083

Loss: 0.567 | Acc: 79.948% (14736/18432)
Loss: 0.567 | Acc: 79.958% (14789/18496)
Loss: 0.567 | Acc: 79.973% (14843/18560)
Loss: 0.567 | Acc: 79.967% (14893/18624)
Loss: 0.567 | Acc: 79.987% (14948/18688)
Loss: 0.567 | Acc: 80.002% (15002/18752)
Loss: 0.567 | Acc: 80.012% (15055/18816)
Loss: 0.567 | Acc: 80.026% (15109/18880)
Loss: 0.566 | Acc: 80.041% (15163/18944)
Loss: 0.566 | Acc: 80.045% (15215/19008)


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/home/fw/anaconda3/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/fw/anaconda3/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/fw/anaconda3/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject